## Agent-based modeling and simulation of transport flows in Amsterdam
### This notebook handles the preprocessing of GVB data to deduce a population that can be used in an agent-based simulation of public transport in Amsterdam.

#### Load all packages

In [1]:
import networkx as nx
import warnings
import datetime
import seaborn as sn
sn.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

#### Load data for origin and destination per hour

In [2]:
origin = pd.read_csv('../../Data/20190403/Datalab_Reis_Herkomst_Uur_20190403.csv', sep=';')
origin = origin.dropna()
destination = pd.read_csv('../../Data/20190403/Datalab_Reis_Bestemming_Uur_20190402.csv', sep=';')
destination = destination.dropna()

In [3]:
# List of all stations in the sub-network
stations = ['Amsteldijk','Amstelstation','Amstelveenseweg','Beethovenstraat','Bullewijk','Burg.de Vlugtlaan', 
    'Centraal Station','Cornelis Troostplein','Dam','De Boelelaan/VU','De Pijp','Europaplein',
    'Gaasperplas','Gein','Gerrit v.d. Veenstraat','Heemstedestraat','Henk Sneevlietweg','IJsbaanpad', 
    'Isolatorweg','Jan v.Galenstraat','Keizersgracht','Koningsplein','Kraaienneststation','Leidseplein',
    'Maasstraat','Marie Heinekenplein','Minervaplein','Muntplein','Museumplein','Nieuwezijds Kolk',
    'Nieuwmarkt','Noord','Noorderpark','Olympiaplein','Olympiaweg','Olympisch stadion','Overamstel', 
    'Postjesweg','Prinsengracht','Reigersbos','Rijksmuseum','Roelof Hartplein','Rokin','Scheldestraat',
    'Spaklerweg','Spui','Station Bijlmer ArenA','Station Diemen-Zuid','Station Duivendrecht', 'Stadionweg'
    'Station Ganzenhoef','Station Holendrecht','Station Lelylaan','Station RAI','Station Sloterdijk', 
    'Strandvliet','Van Baerlestraat','Van der Madeweg','Venserpolder','Verrijn Stuartweg','Victorieplein', 
    'Vijzelgracht','VU medisch centrum','Waalstraat','Waterlooplein','Weesperplein','Wibautstraat','Station Zuid']

#### Change dates to datetime objects and add weekday column

In [4]:
origin['Datum'] = origin['Datum'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%m/%d/%Y %H:%M:%S'))
origin['Weekdag'] = origin['Datum'].apply(lambda x: x.isoweekday())

destination['Datum'] = destination['Datum'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%m/%d/%Y %H:%M:%S'))
destination['Weekdag'] = destination['Datum'].apply(lambda x: x.isoweekday())

#### Remove data where the Noord/Zuid line did not run yet
The Noord/Zuid line was opened on the 21st of July, so all data before August is not used.

In [5]:
origin = origin[(origin['Datum'] >= datetime.datetime(2018, 8, 6)) & 
                 (origin['Datum'] < datetime.datetime(2019, 3, 25))]
destination = destination[(destination['Datum'] >= datetime.datetime(2018, 8, 6)) &
                          (destination['Datum'] < datetime.datetime(2019, 3, 25))]

#### Apply necessary filters for analysis
The first analysis is only applied on one type of day (1 = Mondays) and for the stations that are included in the sub-network.

In [6]:
origin = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'].isin([1, 2, 3, 4]))]
destination = destination[(destination['AankomstHalteNaam'].isin(stations)) & 
                          (destination['Weekdag'].isin([1, 2, 3, 4]))]

#### Remove 'HalteCodes' that are not used in the sub-network

In [7]:
stopdict = {'ITW': 2, 'GN': 2, 'RAI': 2, 'VSW': 1, 'ND': 1, 'RKN': 1, 
            'WPP': 3, 'RGB': 2, 'HVW': 2, 'OAS': 2, 'CLL': 2, 'PJW': 2, 
            'BLW': 2, 'WBS': 3, 'HLD': 2, 'MDW': 0.6666666666666666, 'SVT': 2, 'DMZ': 1, 
            'BMR': 2, 'EPP': 1, 'WTC': 0.6666666666666666, 'DPP': 1, 'JLS': 2, 'DVD': 2, 
            'GZH': 1, 'NDP': 1, 'NMT': 3, 'VLN': 2, 'ASA': 3, 'KEN': 1, 
            'VPD': 1, 'SLW': 3, 'WLP': 3, 'HDS': 2, 'GPP': 1, 'VZG': 1, 
            'CS': 3, 'ASW': 2, 'STD': 2, '08070': 1, '09122': 1, 
            '09121': 1, '07094': 1, '07091': 1, '08071': 1, '09013': 1, 
            '07004': 1, '08076': 1, '08078': 1, '07307': 1, '07308': 1, 
            '07120': 2, '07108': 2, '09083': 2, '09082': 2, '07202': 2, 
            '07121': 2, '09078': 2, '09079': 2, '07084': 2, '07083': 2, 
            '07207': 2, '07205': 2, '07017': 2, '07303': 2, '07350': 2, 
            '09074': 2, '09075': 2, '07098': 2, '07097': 2, '07493': 2, 
            '07490': 2, '09119': 2, '09120': 2, '05036': 3, '05035': 3, 
            '06072': 3, '06071': 3, '06075': 3, '06076': 3, '06073': 3, 
            '06074': 3, '07015': 3, '09049': 3, '09051': 3, '05062': 3, 
            '05061': 3, '07006': 3, '07007': 3, '07322': 3, '07321': 3, 
            '07323': 3, '07324': 3, '06067': 3, '05063': 3, '05032': 3, 
            '05031': 3, '07016': 0.6666666666666666, '09114': 0.6666666666666666, 
            '09111': 0.6666666666666666, '09139': 4, '09140': 4, '05022': 4, 
            '05011': 5, '06089': 5, '05065': 5, '05070': 5, '05069': 5}

In [8]:
origin = origin[origin['VertrekHalteCode'].isin(stopdict.keys())]
origin['AantalLijnen'] = origin['VertrekHalteCode'].map(stopdict)
origin['AantalReizenNorm'] = origin['AantalReizen']/origin['AantalLijnen']

destination = destination[destination['AankomstHalteCode'].isin(stopdict.keys())]
destination['AantalLijnen'] = destination['AankomstHalteCode'].map(stopdict)
destination['AantalReizenNorm'] = destination['AantalReizen']/destination['AantalLijnen']

In [11]:
set(destination['Datum'].tolist())

,Datum,UurgroepOmschrijving (van aankomst),AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalReizen,Weekdag,AantalLijnen,AantalReizenNorm
1746684,2018-08-06 12:00:00,00:00 - 00:59,05011,Centraal Station,4.899218,52.378108,113,1,5.0,22.600000
1746685,2018-08-06 12:00:00,00:00 - 00:59,05022,Centraal Station,4.901043,52.377720,77,1,4.0,19.250000
1746686,2018-08-06 12:00:00,00:00 - 00:59,05032,Dam,4.895494,52.374929,20,1,3.0,6.666667
1746687,2018-08-06 12:00:00,00:00 - 00:59,05036,Rokin,4.892305,52.369855,14,1,3.0,4.666667
1746689,2018-08-06 12:00:00,00:00 - 00:59,05063,Dam,4.890646,52.372446,36,1,3.0,12.000000


#### Group by hour for further analysis

In [69]:
origingrouped = origin[['UurgroepOmschrijving (van vertrek)','VertrekHalteNaam', 'AantalReizenNorm']]
origingrouped = origingrouped.groupby(['UurgroepOmschrijving (van vertrek)', 
                                'VertrekHalteNaam']).aggregate(['min', np.median, np.sum, max])

origingrouped['AantalReizenNorm','mean'] = origingrouped['AantalReizenNorm']['sum'].apply(lambda x: float(x/days))
origingrouped = origingrouped.reset_index()
origingrouped.columns = ['hour', 'stop', 'min', 'median', 'sum', 'max', 'mean']

In [70]:
destinationgrouped = destination[['UurgroepOmschrijving (van aankomst)','AankomstHalteNaam', 'AantalReizenNorm']]
destinationgrouped = destinationgrouped.groupby(['UurgroepOmschrijving (van aankomst)', 
                                'AankomstHalteNaam']).aggregate(['min', np.median, np.sum, 'max'])

destinationgrouped['AantalReizenNorm','mean'] = destinationgrouped['AantalReizenNorm']['sum'].apply(lambda x: float(x/days))
destinationgrouped = destinationgrouped.reset_index()
destinationgrouped.columns = ['hour', 'stop', 'min', 'median', 'sum', 'max', 'mean']

In [71]:
origingrouped.to_csv('../Data/originzondag.csv', index=False)
destinationgrouped.to_csv('../Data/destinationzondag.csv', index=False)

In [83]:
import itertools
comb = list(itertools.combinations(['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag'], 2))
print(comb)

[('maandag', 'dinsdag'), ('maandag', 'woensdag'), ('maandag', 'donderdag'), ('maandag', 'vrijdag'), ('maandag', 'zaterdag'), ('maandag', 'zondag'), ('dinsdag', 'woensdag'), ('dinsdag', 'donderdag'), ('dinsdag', 'vrijdag'), ('dinsdag', 'zaterdag'), ('dinsdag', 'zondag'), ('woensdag', 'donderdag'), ('woensdag', 'vrijdag'), ('woensdag', 'zaterdag'), ('woensdag', 'zondag'), ('donderdag', 'vrijdag'), ('donderdag', 'zaterdag'), ('donderdag', 'zondag'), ('vrijdag', 'zaterdag'), ('vrijdag', 'zondag'), ('zaterdag', 'zondag')]


In [125]:
# gvborigin_maandag, gvborigin_dinsdag
cossimdict = {}

In [155]:
import math

cossimlist = []

def cosine_similarity(vec1,vec2):
        sum11, sum12, sum22 = 0, 0, 0
        for i in range(len(vec1)):
            x = vec1[i]; y = vec2[i]
            sum11 += x*x
            sum22 += y*y
            sum12 += x*y
        return sum12/math.sqrt(sum11*sum22)

for key in gvborigin_maandag.keys():
    try:
        cossimlist.append((key, cosine_similarity(gvbdestination_zaterdag[key][2], gvbdestination_zondag[key][2])))
    except:
        pass
    
cossimdict[('zaterdag', 'zondag')] = cossimlist

In [156]:
cossimdict

{('maandag', 'dinsdag'): [('07', 0.9998524422455223),
  ('11', 0.9989490971864123),
  ('19', 0.9983773934263964),
  ('01', 0.7099427376852885),
  ('08', 0.9997336193531983),
  ('17', 0.9998719944813719),
  ('06', 0.9997449481239412),
  ('05', 0.9911411056052105),
  ('12', 0.9989271782073347),
  ('20', 0.99875179119114),
  ('15', 0.999692903938691),
  ('10', 0.9985226930135487),
  ('02', 0.001253043463130304),
  ('23', 0.9972971562216059),
  ('16', 0.999742586705832),
  ('22', 0.9989532140620976),
  ('18', 0.999487019683943),
  ('21', 0.9991241485160575),
  ('14', 0.9995668100788224),
  ('13', 0.9993170697875785),
  ('04', 0.0),
  ('00', 0.9969144516931623),
  ('09', 0.9989545303935922)],
 ('maandag', 'woensdag'): [('07', 0.9995705153918079),
  ('11', 0.9985068099502112),
  ('19', 0.9963127515271125),
  ('01', 0.9965926536609709),
  ('08', 0.9997608175841137),
  ('17', 0.9982507677374463),
  ('06', 0.9993397237330223),
  ('05', 0.9906221713447082),
  ('12', 0.9990156736172925),
  ('20',

#### Create plot of distribution for a certain hour

In [157]:
cossimdict
import pickle
output = open('../Data/cossim_destination.pkl', 'wb')
pickle.dump(cossimdict, output)
output.close()

In [ ]:
# pkl_file = open('myfile.pkl', 'rb')
# mydict2 = pickle.load(pkl_file)
# pkl_file.close()